In [1]:
import csv
import exrex # !pip install exrex
import json 
import pandas as pd
import numpy as np
import re 
import spacy

from tqdm import tqdm
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split

# ebablbe auto-completion
%config Completer.use_jedi = False

## Load data

In [2]:
def show_entity(text_entities, pattern_matchs):
    for example in text_entities:
        print('-'*50)
        text, annotations = example
        print(text) 
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')
    print('='*50)
    print(*pattern_matchs, sep='\n')
    
options = {'colors': {'ACTION':"#56D7C4", 'PART_NUM':"#92E0AA"} }

def show_text(text, annotations=None, options=options, show_detail=True, standardize_text=None):
    if standardize_text is not None:
        text = standardize_text(text)
    doc = nlp(text)
    print('='*100)
    if show_detail:
        print(doc.text) 
    spacy.displacy.render(doc, style='ent', options=options)
    if show_detail and annotations is not None and len(annotations['entities'])>0:
        print('-'*25, 'acutal entities', '-'*25)
        print(*[(text[start:end], start, end, label) for start, end, label in annotations['entities']], sep='\n')   
    if show_detail and len(doc.ents)>0:
        print('-'*25, 'predict entities', '-'*25)
        print(*[(ent.text, ent.start_char, ent.end_char, ent.label_) for ent in doc.ents], sep='\n')      

def show_texts(data, indexes=None, options=options, show_detail=True, standardize_text=None):
    def show_text_(data):
        if isinstance(data, str):
            show_text(data, None, options=options, show_detail=show_detail, standardize_text=standardize_text)
        else:
            text, annotations = data
            show_text(text, annotations, options=options, show_detail=show_detail, standardize_text=standardize_text)  
            
    if indexes is not None:
        for i in indexes:
            show_text_(data[i])   
    else:
        for one_data in data: 
            show_text_(one_data)         

In [3]:
df_case = pd.read_pickle('./data/df_case.pkl') 
df_train_test = pd.read_pickle('./data/df_train_test.pkl') 

In [98]:
with open('./data/part_num_patterns.json') as json_file:
    part_num_patterns = json.load(json_file)
    
with open('./data/common_words_patterns.json') as json_file:
    common_words_patterns = json.load(json_file)
    
with open('./data/texts.json') as json_file:
    texts = json.load(json_file)    
    
with open('./data/train_texts.json') as json_file:
    train_texts = json.load(json_file)
    
with open('./data/test_texts.json') as json_file:
    test_texts = json.load(json_file)    
    
with open('./data/train_text_entities.json') as json_file:
    train_text_entities = json.load(json_file)
    
with open('./data/test_text_entities.json') as json_file:
    test_text_entities = json.load(json_file)    
    

## Train the model

In [71]:
!python -m spacy train config_gpu_aug.cfg --output ./output/gpu_aug --paths.train ./data/train_aug.spacy --paths.dev ./data/test.spacy --gpu-id 0

2021-08-25 09:31:08.288010: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-25 09:31:11,571] [INFO] Set up nlp object from config
[2021-08-25 09:31:11,588] [INFO] Pipeline: ['transformer', 'ner']
[2021-08-25 09:31:11,595] [INFO] Created vocabulary
[2021-08-25 09:31:11,596] [INFO] Finished initializing nlp object
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are init

## Evaluate

###  Check test data

In [73]:
!python -m spacy evaluate output/gpu_aug/model-best ./data/test.spacy --gpu-id 0

2021-08-25 10:13:52.201493: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   97.15 
NER R   100.00
NER F   98.55 
SPEED   4284  


=============================== NER (per type) ===============================

               P        R       F
PART_NUM   97.15   100.00   98.55



In [74]:
!python -m spacy evaluate output/gpu_aug/model-best ./data/test_aug.spacy --gpu-id 0

2021-08-25 10:14:57.815083: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   98.23 
NER R   99.93 
NER F   99.07 
SPEED   3556  


=============================== NER (per type) ===============================

               P       R       F
PART_NUM   98.23   99.93   99.07



In [75]:
nlp = spacy.load("output/gpu_aug/model-best") 

In [80]:
indexes = np.random.randint(0, len(test_text_entities), 10)
show_texts(test_data, indexes) 

please reassign to william niedecken


/usr/local/lib/python3.6/dist-packages/spacy/displacy/__init__.py:191: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)


please order the following item and ship to my hfpu in west boylston . adf unit , cn598-67008 ( 1 ) . #normal


------------------------- acutal entities -------------------------
('cn598-67008', 82, 93, 'PART_NUM')
------------------------- predict entities -------------------------
('cn598-67008', 82, 93, 'PART_NUM')
revisit 7/14


i need to return to site


...

please create clone for revisit and move part to new wo . thank you . . please order . . 1 ea . . . . rm2-7458-010cn dc controller . . . nbd . . . hal . 


------------------------- acutal entities -------------------------
('rm2-7458-010cn', 102, 116, 'PART_NUM')
------------------------- predict entities -------------------------
('rm2-7458-010cn', 102, 116, 'PART_NUM')
part is still in transit


please order l15809-001 docking station . 


------------------------- acutal entities -------------------------
('l15809-001', 13, 23, 'PART_NUM')
------------------------- predict entities -------------------------
('l15809-001', 13, 23, 'PART_NUM')
please order the following parts and ship to my fedex in brentwood . g1w39-67935 scanner control board for an e58650 note that the previous one i ordered came as a physically different part . please make sure part fits into an hp e58650 . - csutter


------------------------- acutal entities -------------------------
('g1w39-67935', 69, 80, 'PART_NUM')
------------------------- predict entities -------------------------
('g1w39-67935', 69, 80, 'PART_NUM')


In [81]:
show_texts(test_text_entities, indexes, show_detail=False) 

### Check Error Data

In [82]:
def get_error_data(data):
    error_data = []
    for i in range(len(data)):
        text, annotations = data[i]
        doc = nlp(text)
        entities = annotations['entities']
        if len(entities) != len(doc.ents):
            error_data.append(data[i])
            print(i, len(error_data))
        else:
            exit_flag = False
            actual_entities = {(start, end, label): label for start, end, label in entities}
            predict_entities = {(ent.start_char, ent.end_char, ent.label_): ent.label_ 
                                for ent in doc.ents}
            
            for key, _ in actual_entities.items():                
                if key not in predict_entities:
                    print(actual_entities, predict_entities, sep='\n') 
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
            if exit_flag: continue
            for key, _ in predict_entities.items():
                if key not in actual_entities:
                    print(actual_entities, predict_entities)
                    error_data.append(data[i])
                    print(i, len(error_data))
                    break
                
    return error_data

error_data = get_error_data(test_text_entities)

56 1
64 2
89 3
95 4
151 5
176 6
181 7
194 8
204 9
243 10
488 11
504 12
519 13
583 14
687 15
689 16
921 17
980 18
1066 19
1206 20
1336 21
1380 22
1422 23
1431 24
1500 25


In [83]:
print(len(error_data))

25


In [84]:
show_texts(error_data)  

please order rm2-1278-000cn ( qty 1 ) and send to hfpu in lockport , il . 3gy10a-cnncmc6337-hp laserjet managed e60165dn printer case: 5067812295 wo-014252961 po119240397 0645153938 neovia 3930 cedar creek dr joliet , il . 60436 larry heagle 1815-768-3282 alt: 779-702-0498 rm2-1278-000cn assembly-door link e601xx 51771292


------------------------- acutal entities -------------------------
('rm2-1278-000cn', 13, 27, 'PART_NUM')
('rm2-1278-000cn', 274, 288, 'PART_NUM')
------------------------- predict entities -------------------------
('rm2-1278-000cn', 13, 27, 'PART_NUM')
('1815-768-3282', 242, 255, 'PART_NUM')
('779-702-0498', 261, 273, 'PART_NUM')
('rm2-1278-000cn', 274, 288, 'PART_NUM')
please send maintenance kit fixing assembly ( 110v ) rm2-5476-000 1 . #normal part not available checking part #


------------------------- predict entities -------------------------
('rm2-5476-000', 53, 65, 'PART_NUM')
please order part# j8j93–67901 nbd to address on work order


------------------------- predict entities -------------------------
('j8j93–67901 nbd', 19, 34, 'PART_NUM')
order 1ea b5l47-67018 b5l47-67019 . part on b-o


------------------------- acutal entities -------------------------
('b5l47-67018', 10, 21, 'PART_NUM')
('b5l47-67019', 22, 33, 'PART_NUM')
------------------------- predict entities -------------------------
('b5l47-67018', 10, 21, 'PART_NUM')
('b5l47-67019', 22, 33, 'PART_NUM')
('b-o', 44, 47, 'PART_NUM')
#normal printer have error code 50 . 3f . 11 high temperature error 1 contacted tech assist for a new action plan . to replace rl1-4005-000 fuser power supply assembly


...

------------------------- acutal entities -------------------------
('l41606-011', 67, 77, 'PART_NUM')
('5851-7913', 185, 194, 'PART_NUM')
------------------------- predict entities -------------------------
('l41606-011', 67, 77, 'PART_NUM')
('l4606-011', 164, 173, 'PART_NUM')
('5851-7913', 185, 194, 'PART_NUM')
please send parts to 800 main st . paterson nj 07503 attn: nick lavcanski ( 973 ) 202-1231 jc98-05425a qty:2


------------------------- acutal entities -------------------------
('jc98-05425a', 91, 102, 'PART_NUM')
------------------------- predict entities -------------------------
('202-1231', 82, 90, 'PART_NUM')
('jc98-05425a', 91, 102, 'PART_NUM')
please fedex a t030-67901 , t0b28-67901 and a t0b27-67901 to: 16633 schoenborn st north hills , ca 91343


------------------------- acutal entities -------------------------
('t0b28-67901', 28, 39, 'PART_NUM')
('t0b27-67901', 46, 57, 'PART_NUM')
------------------------- predict entities -------------------------
('t030-67901', 15, 25, 'PART_NUM')
('t0b28-67901', 28, 39, 'PART_NUM')
('t0b27-67901', 46, 57, 'PART_NUM')
please add these parts and order replacements to my trunk stock: either one each: pn rm2-1275-000 pn rm2-6772_000 or one pnj7z98-67902 . wells fargo 07/20/2021 9:32 pm cst - parts confirmed as shipped . closed the ticket . 07/16/2021 8:51 pm cst - call is completed - on break until my replacement trunk stock ships and i have the tracking numbers . 07/16/2021 4:48 pm cst *fixed* tray 2 jamming


------------------------- predict entities -------------------------
('rm2-1275-000', 85, 97, 'PART_NUM')
lex-40x-0100 , 2 , c2etes26


------------------------- predict entities -------------------------
('lex-40x-0100', 0, 12, 'PART_NUM')


### Some Features and Issues

#### Miss some part-numbers 

In [86]:
sample_parts = ['M14302-161', '926537-001', 'M46915-D01', '140314-FB2', '176253-8B0', 
               '442010-ABC', 'L07896-DB1', '168757-B21', 'FE-15520-01', 'RM1-7867-000CN', 
               '5066-4719', '5066-3872-1', '3HZ90-30001', 'T6M22-30001', '98544-04105', 
               'C8109-69018', 'L26480-DB1', '632427-001B', 'CN463-69003', 'RM1-1298BULK', 
               '8-752-078-46', 'X-4035-119-1', 'TBLB3002-A04', '4822-701-15319', '08-2212C-33W', 
               '19-40065-011', 'ATT09850-66532', '98564-66572-SQ', '897-250144AA', '738-576-94', 
               '1FQ6-0001', '1818-7469-LGSA', '1820-6361M', '1818-6868-SAM', '3138-107-96160', 
               '686858R-999', 'MD30C-A2', '29-26196-00', 'DEL-7D092', 'LEX-40X2384', 'LEX-40X2831H', 
               'SAM-6107001172', 'SHA-1625DS51', 'XER-006R01275', 'BRO-TN115BK', 'DEL-310-5811', 
               'DEL-PK496', 'KYO-1702F97US0', 'LA95-CA', 'KYO-DK-310H', 'EPS-C31CD38A9921', 
               'KYO-F994091H', 'EPS-CEPS-003G', 'LEX-24015SA', 'LEX-62D1X00', 'LEX-72K0DV0', 
               'LEX-801HC', 'LEX-C5220CSH', 'TRO-78-24619-001', 'LEX-C544X1CG', 'HPE-749797-001', 
               'SFT-P-895', 'LEX-E260A21A-C', 'RIC-D0296509', 'SEA-ST1000LM049', 'LEX-12A8400-PC', 
               'HPE-JL258A', 'RIC-400507', 'FUJ-FPCPR362AQ', 'TOS-6LE8296100', 'XER-003K04980', 
               'XER-116111500', 'ATS-2711FXSC-901', 'BRO-TN336BK', 'DEL-310-5807', 'HPI-CF258A-M', 
               'LEX-T101-0000000', 'DEL-9PN5P', 'SAM-BA92-08880A', 'APC-RBC7', 'KMH-4448-121', 
               'SXC-C7ABTTAAB', 'TRO-04621201', 'VEF-28924-04-R', 'WFP-HPM527Z', 'KYO-1702LK0UN2', 
               'APC-SUA1500RM2U', 'KYO-302NM18021', 'HPE-JH329-61001', 'KYO-1T02GA0US0', 'LEB-04X4674',
               'KYO-302GR93034', 'DEL-V1RX3', 'LEX-40X0593', 'TOS-6LH3460800', 'RIC-AE020171', 
               'XER-59K60140', 'BRO-LM5140001', 'KMH-A00JA56600', 'KYO-302F906240', 'BRO-LEM084001', 
               'XER-116-2035', 'BRO-BU100CLH', '3X-LK465-A2', '4G1-3999-030CN', 'H3980-60002BULK', 
               'SHA-VHI0MFP000', 'TOS-C017839000', 'JC66-01190A', 'XER-3335DNI', 'XER-3635MFP-S', 
               'XER-4622DN', 'XER-7750-Z2', 'DEL-D1320-W3', 'DEL-P637D', 'LEX-C540X35G', 
               'SHA-CPLTM8190F', 'SHACFRM-1380DS53', 'XER-401-0855-0', '90-0077', 'BRO-LF6710001', 
               'Q3938-68001-PCA', 'C314X+49A-90002', '20ER-84530KC', 'RM2-2903-BULK', 'CE506-67919-BU', 
               'CH971-91596', 'DEL-FM235', 'JC44-00210E', 'SHACFRM-1509DS52', 'SHADHAI-5043FCPZ', 
               'SHAGCAB-1102FCB1', '000-50-02-008', '504000226-DD', 'SHAVHPLG217L5A-1', 'SHAVHPOSH03Y02-1', 
               'SHADUNT-8962FCP1', 'SHAKI-OK0010FCPZ', 'SHAQSW-P0506FCZZ', 'SHAVHPGP1A73AR-1', 'SHARH-IX0055FCPZ', 
               '358-000001', 'SHAVHPGP1A71L3-18', 'B23-0994', '504000226-D', '5066-0731', 
               'E0-CABLE-01', 'RM1-8508-010-BU', 'B3M77-67902-BU', 'CA02626-E029FJ', 'CF065-67901-BU', 
               'C2H57-67901-BULK', 'LN08-A2', '693709-001B', 'SS467-67001', 'LEX-70C0D20', 
               'SHAQSW-M0518FCPZ', 'SHALX-BZ0994FCPZ', 'LN08X-TF', 'SHAGCAB-1507FCAZ']

sample_parts = [part.lower() for part in sample_parts]

In [87]:
def get_texts(parts, standardize_text=standardize_text1):
    texts = [standardize_text(f'Please order qty 1 {part} NBD to my HAL in Lenexa. Thank you.') for part in parts]
    return texts

def check_sample_parts(parts):  
    texts = get_texts(parts)
    docs = nlp.pipe(texts)
    texts = []
    error_texts = []
    errpr_parts = []
    for doc, part in zip(docs, parts):
        texts.append(doc.text)
        if len(doc.ents) !=1 or doc.ents[0].text != part.lower():
            error_texts.append(doc.text)
            errpr_parts.append(part)
    return texts, error_texts, errpr_parts
        
parts = ['000-50-02-008']
texts, error_texts, errpr_parts = check_sample_parts(parts)
show_texts(texts, show_detail=True) 

please order qty 1 000-50-02-008 nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('000-50-02-008', 19, 32, 'PART_NUM')


In [89]:
parts = sample_parts
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')
show_texts(error_texts, show_detail=True)  

find 2/160 errors
please order qty 1 lex-c544x1cg nbd to my hal in lenexa . thank you . 


------------------------- predict entities -------------------------
('lex-c544x1cg nbd', 19, 35, 'PART_NUM')
please order qty 1 c314x+49a-90002 nbd to my hal in lenexa . thank you . 


In [96]:
error_text_entities, error_pattern_matchs = generate_label(error_texts, patterns=patterns)
show_entity(error_text_entities, error_pattern_matchs)

('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d[a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d', 0)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d-\\d', 0)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z]', 0)
('[a-zA-Z][a-zA-Z]\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]', 0)
('\\d-\\d\\d\\d-\\d\\d\\d-\\d\\d', 0)
('[a-zA-Z]-\\d\\d\\d\\d-\\d\\d\\d-\\d', 0)
('[a-zA-Z][a-zA-Z][a-zA-Z][a-zA-Z]\\d

I think that the issues are mainly caused by lacking of these parts in training data. 

There are 2 solutions:

1. merge the results of regular expressions and the model.
2. use data augmentation to generate a lot of random part numbers based on the rules.

####  Find potentail part number

In [91]:
parts = ['FE-15520-01', 'FE-15520-012']
texts, error_texts, errpr_parts = check_sample_parts(parts)
print(f'find {len(error_texts)}/{len(parts)} errors')



find 0/2 errors


In [95]:
some_text_entities, some__pattern_matchs = generate_label(texts, patterns=patterns)
show_entity(some_text_entities, some__pattern_matchs)


--------------------------------------------------
please order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 
('fe-15520-01', 19, 30, 'PART_NUM')
--------------------------------------------------
please order qty 1 fe-15520-012 nbd to my hal in lenexa . thank you . 

('[a-zA-Z][a-zA-Z]-\\d\\d\\d\\d\\d-\\d\\d', 1)
('[a-zA-Z]\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-\\d[a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z][a-zA-Z]', 0)
('[a-zA-Z]\\d\\d\\d\\d\\d-[a-zA-Z][a-zA-Z]\\d', 0)
('\\d\\d\\d\\d\\d\\d-[a-zA-Z]\\d\\d', 0)
('[a-zA-Z][a-zA-Z]\\d-\\d\\d\\d\\d-\\d\\d\\d[a-zA-Z][a-zA-Z]', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d-\\d\\d\\d\\d-\\d', 0)
('\\d[a-zA-Z][a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d[a-zA-Z]\\d\\d-\\d\\d\\d\\d\\d', 0)
('\\d\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('[a-zA-Z]\\d\\d\\d\\d-\\d\\d\\d\\d\\d', 0)
('\

#### find potential actions 

it has some effects, but not good enough.

In [65]:
texts = ['order qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'orders qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'send qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'sends qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ship qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . ',
         'ships qty 1 fe-15520-01 nbd to my hal in lenexa . thank you . 2012-02-02',         
        ]

show_texts(texts, show_detail=False) 

## Output Results

In [ ]:
common_words_patterns = [
 {'expression': r'order', 'label': 'ACTION'}, 
 {'expression': r'recommended', 'label': 'ACTION'}, 
 {'expression': r'hfpu', 'label': 'ACTION'}, 
 {'expression': r'ship', 'label': 'ACTION'}, 
 {'expression': r'fedex', 'label': 'ACTION'}, 
 {'expression': r'hal', 'label': 'ACTION'}, 
 {'expression': r'site', 'label': 'ACTION'}, 
 {'expression': r'send', 'label': 'ACTION'}, 
 {'expression': r'location', 'label': 'ACTION'}, 
 {'expression': r'pickup', 'label': 'ACTION'}, 
 {'expression': r'sent', 'label': 'ACTION'}, 
 {'expression': r'requested', 'label': 'ACTION'}, 
 {'expression': r'reorder', 'label': 'ACTION'}, 
 {'expression': r'street', 'label': 'ACTION'}, 
 {'expression': r'hfp', 'label': 'ACTION'}, 
 {'expression': r'shipped', 'label': 'ACTION'}, 
 {'expression': r'delivery', 'label': 'ACTION'}, 
 {'expression': r'recommend', 'label': 'ACTION'}, 
 {'expression': r'warehouse', 'label': 'ACTION'}   
]

6110

In [110]:
def get_part_nums(texts):
    docs = nlp.pipe(texts)
    part_nums = []
    for doc in docs:
        doc_part_nums = [ent.text for ent in doc.ents]
        part_nums.append(doc_part_nums)    
        
    return part_nums 

part_nums = get_part_nums(texts[0:100])
part_nums

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 ['ch538-67076',
  't8w16-67006',
  'ch538-67018',
  'ck839-67005',
  'cn727-67023',
  'w6b55-67007'],
 [],
 ['l98769-003'],
 ['l98769-003'],
 [],
 [],
 ['l51580-001', 'l31365-001'],
 [],
 [],
 ['m21823-601'],
 [],
 ['933321-855'],
 [],
 [],
 [],
 ['l62781-001', 'l63364-001'],
 [],
 ['a8p79-65'],
 [],
 [],
 ['m12882-601'],
 [],
 [],
 [],
 ['917047-001'],
 [],
 [],
 ['j7z09-67965', 'j7z09-67970'],
 [],
 ['p1b11-67901'],
 ['p1b11-67901'],
 [],
 [],
 ['932688-601'],
 ['l27294-001'],
 ['l14370-001'],
 ['932688-601'],
 ['l85364-001', 'l13665-601'],
 ['q6718-67017', 'q6718-67025'],
 ['932688-601'],
 ['932688-601'],
 ['932688-601'],
 ['932688-601'],
 [],
 [],
 [],
 ['x3a92-60003'],
 [],
 [],
 [],
 ['rm2-6576-000cn', 'rk2-6027-000cn', 'rm2-0077-000cn'],
 ['rm2-5683-000cn', 'rm2-2585-000cn'],
 [],
 [],
 [],
 [],
 [],
 ['m11775-601', 'm11775-601'],
 ['m07092-001'],
 [],
 ['923771-601'],
 [],
 ['j8a10-60108', 'j8a10-60110', 'rm2-8421-000cn', 'j8a1

In [107]:
texts[0:10]

['#normal part escalated',
 'please cancel backordered part . customer has canceled service',
 'according to bryan this laptop shows repair completed and that it is already rerurned to the end user in their records . ',
 'back order part',
 'please send this case to corey harbarenko . corey is aware and is covering for me while i am on vacation . thanks',
 'originally went onsite and replaced the facia unable to get it to sit properly . contacted support mike . recommended possible depot of customer approves . spoke with the customer mike he accepted depot repair . contacted support awaiting new wo number they . 6/13 left vm for customer . -kevin 6/13 please reorder previously ordered part and ship to my hfpu-kevin 6/19 left vm to schedule awaiting call back-kevin',
 'originally went onsite and replaced the facia unable to get it to sit properly . contacted support mike . recommended possible depot of customer approves . spoke with the customer mike he accepted depot repair . contacted